In [ ]:
#source : https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset?resource=download

In [21]:
import os
import glob
import pandas as pd

# Specify the directory where your .csv files are located
directory = 'oldDataset'

# Method to get all .csv files in the directory
csv_files = glob.glob(f"{directory}/*.csv")

# Display the names of all .csv files
print("List of CSV files found:")
for file in csv_files:
    print(file)

# Define a list to hold DataFrames
dataframes = []

# Define a reference for columns to ensure consistency
reference_columns = None

# Process each .csv file
for file in csv_files:
    try:
        # Load only the first 100 rows
        df = pd.read_csv(file, nrows=100)
        
        # Skip the file if it's empty
        if df.empty:
            print(f"Skipping '{file}' because it is empty.")
            continue
        
        # Check column consistency
        if reference_columns is None:
            # Set reference columns based on the first valid file
            reference_columns = df.columns.tolist()
        else:
            # If the current file columns do not match the reference, skip it
            if df.columns.tolist() != reference_columns:
                print(f"Skipping '{file}' due to column mismatch.")
                continue
        
        # Append the valid DataFrame to the list
        dataframes.append(df)
        print(f"Included '{file}' with shape {df.shape}.")

    except Exception as e:
        print(f"Error processing '{file}': {e}")

# Merge all DataFrames into one if there are any valid files
if dataframes:
    merged_df = pd.concat(dataframes, ignore_index=True)
    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv('merged_dataset.csv', index=False)
    print(f"\nAll valid CSV files merged successfully. Saved as 'merged_dataset.csv' with shape {merged_df.shape}.")
else:
    print("No valid CSV files found to merge.")


List of CSV files found:
oldDataset\Air Conditioners.csv
oldDataset\All Appliances.csv
oldDataset\All Books.csv
oldDataset\All Car and Motorbike Products.csv
oldDataset\All Electronics.csv
oldDataset\All English.csv
oldDataset\All Exercise and Fitness.csv
oldDataset\All Grocery and Gourmet Foods.csv
oldDataset\All Hindi.csv
oldDataset\All Home and Kitchen.csv
oldDataset\All Movies and TV Shows.csv
oldDataset\All Music.csv
oldDataset\All Pet Supplies.csv
oldDataset\All Sports Fitness and Outdoors.csv
oldDataset\All Video Games.csv
oldDataset\Amazon Fashion.csv
oldDataset\Amazon Pharmacy.csv
oldDataset\Amazon-Products.csv
oldDataset\Baby Bath Skin and Grooming.csv
oldDataset\Baby Fashion.csv
oldDataset\Baby Products.csv
oldDataset\Backpacks.csv
oldDataset\Badminton.csv
oldDataset\Bags and Luggage.csv
oldDataset\Ballerinas.csv
oldDataset\Beauty and Grooming.csv
oldDataset\Bedroom Linen.csv
oldDataset\Blu-ray.csv
oldDataset\Camera Accessories.csv
oldDataset\Cameras.csv
oldDataset\Camping a

In [22]:
import pandas as pd

# Load the already merged dataset
merged_file = 'merged_dataset.csv'

try:
    # Read the merged dataset
    df = pd.read_csv(merged_file)
    print(f"Loaded '{merged_file}' with shape {df.shape} and columns: {df.columns.tolist()}")

    # Step 1: Remove unwanted columns if they exist
    columns_to_remove = ['link', 'discount_price']
    df = df.drop(columns=columns_to_remove, errors='ignore')
    
    # Step 2: Define new column names (must match the number of columns after removal)
    new_column_names = ['name', 'category', 'sub_category', 'photo', 'no_of_ratings', 'price']
    
    # Check if the remaining columns match the expected count
    if len(df.columns) == len(new_column_names):
        df.columns = new_column_names
        print(f"Columns renamed successfully to: {df.columns.tolist()}")
    else:
        print(f"Warning: Column count mismatch. Expected {len(new_column_names)} columns but got {len(df.columns)}.")

    # Step 3: Save the cleaned DataFrame to a new CSV file
    cleaned_file = 'cleaned_merged_dataset.csv'
    df.to_csv(cleaned_file, index=False)
    print(f"Cleaned dataset saved as '{cleaned_file}' with shape {df.shape}.")

except FileNotFoundError:
    print(f"Error: The file '{merged_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Loaded 'merged_dataset.csv' with shape (10830, 9) and columns: ['name', 'main_category', 'sub_category', 'image', 'link', 'ratings', 'no_of_ratings', 'discount_price', 'actual_price']
Cleaned dataset saved as 'cleaned_merged_dataset.csv' with shape (10830, 7).


In [23]:
import pandas as pd
import requests

# Load the cleaned merged dataset
merged_file = 'cleaned_merged_dataset.csv'
log_file = 'removed_rows_log.csv'

# Function to check if a URL is valid
def is_valid_url(url):
    try:
        response = requests.head(url, timeout=5)  # Use HEAD request for faster checking
        return response.status_code == 200
    except requests.RequestException:
        return False

# Step 1: Load the dataset
try:
    df = pd.read_csv(merged_file)
    print(f"Loaded '{merged_file}' with shape {df.shape}")

    # Step 2: Initialize a list to store removed rows for logging
    removed_rows = []

    # Step 3: Check each URL in the 'photo' column
    for index, row in df.iterrows():
        photo_url = row['photo']
        if not is_valid_url(photo_url):
            # Log the removed row
            removed_rows.append(row)
            # Drop the row from the DataFrame
            df.drop(index, inplace=True)

    # Step 4: Save the updated DataFrame (with only valid URLs)
    cleaned_file = 'final_cleaned_dataset.csv'
    df.to_csv(cleaned_file, index=False)
    print(f"Filtered dataset saved as '{cleaned_file}' with shape {df.shape}.")

    # Step 5: Save the log of removed rows
    if removed_rows:
        removed_df = pd.DataFrame(removed_rows)
        removed_df.to_csv(log_file, index=False)
        print(f"Log of removed rows saved as '{log_file}' with {len(removed_rows)} entries.")
    else:
        print("No invalid URLs found. No rows removed.")

except FileNotFoundError:
    print(f"Error: The file '{merged_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Loaded 'cleaned_merged_dataset.csv' with shape (10830, 7)
An error occurred: 'photo'


In [27]:
import pandas as pd
import random

# Load the cleaned merged dataset
merged_file = 'final_cleaned_dataset_with_extra_columns.csv'

# Conversion rate from ₹ to USD (update this as per the current rate)
conversion_rate = 0.012  # Example: 1 INR = 0.012 USD

# Function to convert ₹ to USD and ensure the result is float
def convert_to_usd(price_str):
    # Remove the currency symbol and commas
    price_str = price_str.replace('₹', '').replace(',', '').strip()
    
    try:
        # Convert the price string to float
        price_in_inr = float(price_str)
        # Convert INR to USD
        price_in_usd = price_in_inr * conversion_rate
        return round(price_in_usd, 2)  # Ensure result is a float and round to two decimal places
    except ValueError:
        # If there's an error in conversion (e.g., empty or invalid value), return None or handle as needed
        return None

# Function to generate random stock quantity
def generate_random_stock():
    return random.randint(1, 1000)  # Random stock between 1 and 1000

# Function to generate random color
def generate_random_color():
    colors = ['Red', 'Blue', 'Green', 'Yellow', 'Black', 'White', 'Purple', 'Orange', 'Pink']
    return random.choice(colors)  # Randomly choose a color

# Function to extract a random number of keywords from the product name
def extract_keywords(name):
    # Split the name into words
    words = name.split()
    # Randomly choose between 3 and 5 words
    num_keywords = random.randint(3, 5)
    # Ensure we don't exceed the number of available words in the name
    num_keywords = min(num_keywords, len(words))
    
    # Randomly select 'num_keywords' words from anywhere in the list
    start_index = random.randint(0, len(words) - num_keywords)
    return ' '.join(words[start_index:start_index + num_keywords])  # Return the selected keywords

# Step 1: Load the dataset
try:
    df = pd.read_csv(merged_file)
    print(f"Loaded '{merged_file}' with shape {df.shape}")

    # Step 2: Convert the price column from ₹ to USD and ensure it is a float
    df['price'] = df['price'].apply(lambda x: convert_to_usd(str(x)))  # Apply the conversion function
    
    # Remove any rows where the conversion failed (if any)
    df = df.dropna(subset=['price'])

    # Step 3: Add Random Stock, Color, and Keywords columns
    df['stock'] = df['name'].apply(lambda x: generate_random_stock())  # Add random stock
    df['color'] = df['name'].apply(lambda x: generate_random_color())  # Add random color
    df['keywords'] = df['name'].apply(lambda x: extract_keywords(x))  # Extract random keywords from product name

    # Step 4: Add Discount column with random values between 0 and 30
    df['discount'] = [random.uniform(0, 30) for _ in range(len(df))]  # Random discount between 0 and 30

    # Step 5: Ensure 'ratings' is a float and 'no_of_ratings' is an integer
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')  # Convert ratings to float, invalid values become NaN
    df['ratings'] = df['ratings'].fillna(0.0)  # Replace NaN with 0.0 (or any appropriate value)
    
    df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')  # Convert to integer, invalid values become NaN
    df['no_of_ratings'] = df['no_of_ratings'].fillna(0).astype(int)  # Replace NaN with 0 and convert to int

    # Step 6: Save the updated DataFrame (with converted prices, random stock, color, keywords, and discount)
    cleaned_file = 'final_cleaned_dataset_with_all_columns_and_discount.csv'
    df.to_csv(cleaned_file, index=False)
    print(f"Filtered dataset with all columns saved as '{cleaned_file}' with shape {df.shape}.")

except FileNotFoundError:
    print(f"Error: The file '{merged_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Loaded 'final_cleaned_dataset_with_extra_columns.csv' with shape (10738, 10)
Filtered dataset with all columns saved as 'final_cleaned_dataset_with_all_columns_and_discount.csv' with shape (10738, 11).


In [35]:
import pandas as pd
from datetime import datetime
import json
import random

# Load the cleaned merged dataset
merged_file = 'final_cleaned_dataset_with_all_columns_and_discount.csv'

# Function to process and ensure keywords are in an array format
def process_keywords(keywords):
    if isinstance(keywords, str):
        return keywords.split()  # Split the string by spaces into an array of words
    elif isinstance(keywords, list):
        return keywords  # If already an array, return as is
    else:
        return []  # If not a string or list, return an empty array

# Step 1: Load the dataset
try:
    df = pd.read_csv(merged_file)
    print(f"Loaded '{merged_file}' with shape {df.shape}")

    # Step 2: Prepare the MongoDB JSON structure
    json_data = []
    for index, row in df.iterrows():
        product = {
            "name": row['name'],
            "ratings": row['ratings'],
            "keywords": process_keywords(row['keywords']),  # Process and ensure keywords is a list of words
            "no_of_ratings": row['no_of_ratings'],  # Assuming this exists in the dataset
            "description": row['description'] if 'description' in row else "",  # Default empty if no description
            "category": row['category'],
            "discount": round(row['discount'], 2) if 'discount' in row and pd.notnull(row['discount']) else 0.0,
            "price": row['price'],
            "color": row['color'],  # Treat 'color' as a simple string, not an array
            "stock": row['stock'] if 'stock' in row else random.randint(1, 1000),  # Random stock if not available
            "createdAt": datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),  # Set createdAt to current timestamp
            "photo": row['photo']  # Assuming 'photo' column contains correct image path
        }
        json_data.append(product)

    # Step 3: Save the JSON data to a file
    json_filename = 'mongo_import_data.json'
    with open(json_filename, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    print(f"JSON file '{json_filename}' successfully saved for MongoDB import.")

except FileNotFoundError:
    print(f"Error: The file '{merged_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Loaded 'final_cleaned_dataset_with_all_columns_and_discount.csv' with shape (10738, 11)
JSON file 'mongo_import_data.json' successfully saved for MongoDB import.


In [6]:
import pandas as pd
import json

# Load the dataset
csv_file = "final_cleaned_dataset_with_all_columns_and_discount.csv"
df = pd.read_csv(csv_file)

# Set photo column to null (None in Python)
df["photo"] = None

# Extract distinct categories
categories = df["category"].unique().tolist()

# Create a list of dictionaries with the required format
categories_list = [{"nameCategorie": category, "photo": None} for category in categories]

# Save to distinct_categories.json
categories_output_file = "distinct_categories.json"
with open(categories_output_file, "w") as f:
    json.dump(categories_list, f, indent=4)

print(f"Distinct categories saved to {categories_output_file}")


Distinct categories saved to distinct_categories.json
